In [1]:
import numpy as np

In [2]:
def viterbi(states, obs, pi, a, b, O):
    """
    Viterbi algorithm
    
    states : list of possible states
    obs : list of observations
    pi : array of initial probabilities
    a : transition matrix
    b : emission matrix
    O : sequence of observations
    """

    N = [obs.index(o) for o in O]
        
    for i, o in enumerate(N):
        if i == 0:
            delta = [pi*b[:, o]]
            psi = [None]
        else:
            step = np.multiply(b[:, o].reshape(2,1), np.multiply(a, delta[i-1]))
            delta.append(np.amax(step, axis=1))
            psi.append(np.argmax(step, axis=1))
    
    i = np.argmax(delta[-1])
    res = [states[i]]
    for t in range(len(delta)-1, 0, -1):
        i = psi[t][i]
        res = [states[i]] + res
    
    return res

In [12]:
def forward_backward(states, obs, pi, a, b, O):
    """
    Forward-backward algorithm
    
    states : list of possible states
    obs : list of observations
    pi : array of initial probabilities
    a : transition matrix
    b : emission matrix
    O : sequence of observations
    """
    N = [obs.index(o) for o in O]
    
    alpha = None
    for o in N:
        if alpha is None:
            alpha = [pi*b[:, o]]
        else:
            step = np.diag(b[:, o]) @ a @ alpha[-1]
            alpha.append(step)
    
    beta = [np.array([1, 1])]
    for o in reversed(N):
        step = a @ np.diag(b[:, o]) @ beta[0]
        beta.insert(0, step)
    
    res = []
    for alpha_t, beta_t in zip(alpha, beta):
        res.append(alpha_t*beta_t/sum(alpha_t*beta_t))
    
    return res

In [6]:
tests = {'1': {
    'pi': np.array([0.5, 0.5]),
    'a': np.array([[0.8, 0.2], [0.2, 0.8]]),
    'b': np.array([[0.5, 0.5], [0.1, 0.9]]),
    'states': ['1', '2'],
    'obs': ['О', 'Р'],
    'O': 'ОРОРОРООРРРРРРРРРРОООООООО'},
 '2': {
    'pi': np.array([0.5, 0.5]),
    'a': np.array([[0.5, 0.5], [0.5, 0.5]]),
    'b': np.array([[0.5, 0.5], [0.51, 0.49]]),
    'states': ['1', '2'],
    'obs': ['О', 'Р'],
    'O': 'ОРОРОРООРРРРРРРРРРОООООООО'}
}

In [26]:
test = input('Номер теста: ')

Номер теста: 1


In [27]:
vit_res = viterbi(**tests[test])
fb_res = forward_backward(**tests[test])

In [28]:
print('    '.join(vit_res))
print(' '.join([f'{p[0]:0.2f}' for p in fb_res]))
print(' '.join([f'{p[1]:0.2f}' for p in fb_res]))

1    1    1    1    1    1    1    1    2    2    2    2    2    2    2    2    2    2    1    1    1    1    1    1    1    1
0.93 0.61 0.94 0.63 0.94 0.64 0.94 0.95 0.46 0.27 0.19 0.15 0.14 0.13 0.14 0.15 0.19 0.28 0.89 0.96 0.98 0.98 0.98 0.98 0.98 0.97
0.07 0.39 0.06 0.37 0.06 0.36 0.06 0.05 0.54 0.73 0.81 0.85 0.86 0.87 0.86 0.85 0.81 0.72 0.11 0.04 0.02 0.02 0.02 0.02 0.02 0.03
